In [1]:
#Auto reloading 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
#Removing indentation issues:
#!autopep8 --in-place --aggressive --aggressive init_params.py
#!autopep8 --in-place --aggressive --aggressive popconstruct.py
#!autopep8 --in-place --aggressive --aggressive generateepochs.py
#!autopep8 --in-place --aggressive --aggressive qvalues.py

In [3]:
!python setup.py build_ext --inplace

running build_ext
copying build/lib.macosx-10.9-x86_64-3.8/agent_timestep.cpython-38-darwin.so -> 


In [4]:
#Importing scripts:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct as popconstruct
import qvalues as qval
import generateepochs as gen
import generate_stop_dataframe as gen_stop
from agentmatrixinit import *
#from general_init_codeblocks import *
from agent_timestep import timestep_mutator, multitimestep_mutator
import pdb
import pipeline_creation as pl_creat
import mega_loop as ml
import plotting_functions as plt_func
import plotting_helper_functions as plt_help
import matplotlib.pyplot as plt

data_dir = "./Data/"
figure_dir = "./Figures/"

2022-07-23 18:42:30,079	INFO worker.py:726 -- Connecting to existing Ray cluster at address: 192.168.1.56:6379


In [5]:
seed = np.random.randint(0,9999999)
#seed = 184183
print(seed)
np.random.seed(seed)
to_check = np.random.randint(0,50,10)
print(to_check)

8229973
[ 3 37 23 11  2 49 35  3 37  5]


In [6]:
#Create the whole pipeline.
#This pipeline creates three sub-pipelines: 
#rewards pipeline (rsg), q values pipeline (q_val), network pipeline(pl), stop pipeline (stop)

pl = pl_creat.create_main_pipeline()
mega_l = ml.mega_loop

In [7]:
#Adding the timestep loop to the main pipeline
pl.add(mega_l)

## Combined loop

In [ ]:
import pickle 

def saveresults_vars(variable, prefix):
    pickle.dump(variable, open(prefix, 'wb'))
    
def loadresults_vars(prefix):
    return pickle.load(open(prefix, "rb"))

In [ ]:
percentage = []
ampl_valuesSTN = []
ampl_valuesD2 = []
onset_valuesSTN = []
onset_valuesD2 = []

for onsetSTN in np.arange(0,110,10):
    
    for amplitudeSTN in np.arange(0.35, 0.475, 0.025):
        
        for amplitudeD2 in np.arange(0.575, 0.7, 0.025):
    
    
            environment = {
                'params': None, #neuron parameters (init_params.py)
                'pops': None,   #population parameters (init_params.py)
                'receps' : None, #receptor parameters (init_params.py)
                'base' : None, #baseline stimulation parameters (init_params.py)
                'dpmns' : None, #dopamine related parameters (init_params.py)
                'd1' : None, #D1-MSNs population related parameters (init_params.py)
                'd2' : None, #D2-MSNs population related parameters (init_params.py)
                'channels' : pd.DataFrame([["left"],["right"]], columns=['action']), #action channels related parameters (init_params.py)
                #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
                #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
                'newpathways' : None, #connectivity parameters (popconstruct.py)
                'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
                'Q_df':None, #initialized Q-values df (qvalues.py)  
                'n_trials': 100, #number of trials (generateepochs.py)
                'volatility': 1, #frequency of changepoints (generateepochs.py)
                'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
                'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
                'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
                'maxstim': 0.95, # amplitude of the cortical input over base line #0.85
                'sustainedfraction': 0.75,

                #STN
                'stop_signal_present': True,
                'stop_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
                'stop_signal_amplitude': amplitudeSTN, # amplitude of the stop signal over base line
                'stop_signal_onset': onsetSTN, #in ms #antes 200
                'stop_signal_duration' : 100., 
                'stop_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                            #/"any" (channel given the stop signal is chosen randomly)
                                            # [list of channels] == subset of channels given the stop signa

                #D2STR
                'stop_signal_present_2': True,
                'stop_signal_probability_2': 1., #probability of trials that will get the stop signal / list of trial numbers
                'stop_signal_amplitude_2': amplitudeD2, # amplitude of the stop signal over base line
                'stop_signal_onset_2': onsetSTN + 5., #in ms #antes 200
                'stop_signal_duration_2' : 160.,
                'stop_signal_channel_2': "all", #"all" (all channels are given the stop signal) 
                                            #/"any" (channel given the stop signal is chosen randomly)
                                            # [list of channels] == subset of channels given the stop signal

                'decision_channel' : '1', #'all'(all channels)
                                          # '0' : left channel considered 
                                          # '1' : right channel considered
            }                  
            results = cbgt.ExecutionManager(cores=7).run([pl]*1,[environment]*1)

            datatables = cbgt.collateVariable(results,'datatables')
            
            truetrials_number = 0
            stop_number = 0
            for i in datatables[0].index:
                if datatables[0].iloc[i]['decisionduration'] > results[0]['stop_signal_onset']:
                    truetrials_number += 1
                if datatables[0].iloc[i]['decision'] == 'stop':
                    stop_number += 1
            truetrials_array.append(truetrials_number)
            stop_percentage.append(stop_number/truetrials_number)
            popfreqs = cbgt.collateVariable(results,'popfreqs')
            ampl_valuesSTN.append(amplitudeSTN)
            ampl_valuesD2.append(amplitudeD2)
            onset_valuesSTN.append(onsetSTN)
            onset_valuesD2.append(onsetSTN+5)
            cbgt.saveResults(results, data_dir+'stop_data_combinedloops_STNampl'+str(amplitudeSTN)+'_D2ampl_'+str(amplitudeD2)+ '_STNonset_'+str(onsetSTN)+'_D2onset_'+str(onsetD2)+'_seed_'+str(seed),['popfreqs','popdata', 'datatables'])
            #firing_rates, q_df, performance, rt_distribution, total_performance = plt_help.extract_relevant_frames(results,seed)
            #plt_func.plot_fr_marked(firing_rates, amplitude, results[0]['stop_signal_onset'])

In [ ]:
saveresults_vars(stop_percentage, data_dir+'loop_combined_stop_probs'+'_seed'+str(seed))
saveresults_vars(onset_valuesSTN, data_dir+'loop_comb_onsettime_onset_valuesSTN'+'_seed'+str(seed))
saveresults_vars(onset_valuesD2, data_dir+'loop_comb_onsettime_onset_valuesD2'+'_seed'+str(seed))
saveresults_vars(ampl_valuesSTN, data_dir+'loop_comb_ampl_valuesSTN'+'_seed'+str(seed))
saveresults_vars(ampl_valuesD2, data_dir+'loop_comb_ampl_valuesD2'+'_seed'+str(seed))
saveresults_vars(truetrials_array, data_dir+'loop_combined_truetrials_number'+'_seed'+str(seed))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
    
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
p = ax.scatter(ampl_valuesSTN, ampl_valuesD2, 
               linewidths=0, alpha=.9, edgecolor=None, s=100, c=stop_percentage)
cbar = fig.colorbar(p, orientation="horizontal", shrink=0.65)
cbar.set_label('Stopping probability')
ax.set_xlabel('AmplitudeSTN')
ax.set_ylabel('AmplitudeD2')
#ax.set_ylim(-10,100)
#ax.set_zlabel('Stopping performance')
ax.figure.savefig('./Figures/'+'AmplitudeSTN-AmplitudeD2_combinedloop'+'_'+str(seed)+'.png')
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
    
fig = plt.figure(figsize=(10, 10))
ax = fig.gca(projection='3d')
p = ax.scatter(ampl_valuesSTN, ampl_valuesD2, onset_valuesSTN,
               linewidths=0, alpha=.9, edgecolor=None, s = 100, c=stop_percentage)
cbar = fig.colorbar(p, orientation="horizontal", shrink=0.55)
cbar.set_label('Stopping probability')
#ax.view_init(elev=22.),
ax.view_init(elev=22., azim=-60)
ax.set_xlabel('AmplitudeSTN')
ax.set_ylabel('AmplitudeD2')
#ax.set_ylim(0, 100)
ax.set_zlabel('OnsettimeSTN')
plt.savefig('./Figures/'+'AmplitudeSTN-AmplitudeD2-OnsetSTN_3d'+'_'+str(seed)+'.png')                   
plt.show()

## Onset loop

In [8]:
import pickle 

def saveresults_vars(variable, prefix):
    pickle.dump(variable, open(prefix, 'wb'))
    
def loadresults_vars(prefix):
    return pickle.load(open(prefix, "rb"))

In [9]:
stop_percentage_tot = []
stop_percentage_true = []
failed_array = []
truetrials_array = []
onset_values = []

for onset in np.arange(0,140,10):
    
    
    environment = {
        'params': None, #neuron parameters (init_params.py)
        'pops': None,   #population parameters (init_params.py)
        'receps' : None, #receptor parameters (init_params.py)
        'base' : None, #baseline stimulation parameters (init_params.py)
        'dpmns' : None, #dopamine related parameters (init_params.py)
        'd1' : None, #D1-MSNs population related parameters (init_params.py)
        'd2' : None, #D2-MSNs population related parameters (init_params.py)
        'channels' : pd.DataFrame([["left"],["right"]], columns=['action']), #action channels related parameters (init_params.py)
        #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
        #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
        'newpathways' : None, #connectivity parameters (popconstruct.py)
        'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
        'Q_df':None, #initialized Q-values df (qvalues.py)  
        'n_trials': 300, #number of trials (generateepochs.py)
        'volatility': 1, #frequency of changepoints (generateepochs.py)
        'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
        'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
        'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
        'maxstim': 0.95, # amplitude of the cortical input over base line #0.85
        'sustainedfraction': 0.75,

        #STN
        'stop_signal_present': True,
        'stop_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
        'stop_signal_amplitude': 0.43, # amplitude of the stop signal over base line
        'stop_signal_onset': onset, #in ms #antes 200
        'stop_signal_duration' : 160., 
        'stop_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                    #/"any" (channel given the stop signal is chosen randomly)
                                    # [list of channels] == subset of channels given the stop signa

        #D2STR
        'stop_signal_present_2': True,
        'stop_signal_probability_2': 1., #probability of trials that will get the stop signal / list of trial numbers
        'stop_signal_amplitude_2': 0.9, # amplitude of the stop signal over base line
        'stop_signal_onset_2': onset, #in ms #antes 200
        'stop_signal_duration_2' : 160.,
        'stop_signal_channel_2': "all", #"all" (all channels are given the stop signal) 
                                    #/"any" (channel given the stop signal is chosen randomly)
                                    # [list of channels] == subset of channels given the stop signal

        'decision_channel' : '1', #'all'(all channels)
                                  # '0' : left channel considered 
                                  # '1' : right channel considered
    }                  
    results = cbgt.ExecutionManager(cores=7).run([pl]*1,[environment]*1)

    datatables = cbgt.collateVariable(results,'datatables')
    stop_number = 0
    failed_number = 0
    truetrials_number = 0
    
    for i in datatables[0].index:
        if datatables[0].iloc[i]['decisionduration'] > results[0]['stop_signal_onset']:
            truetrials_number += 1
        if datatables[0].iloc[i]['decision'] == 'stop':
            stop_number += 1
        if datatables[0].iloc[i]['decision'] != 'stop':
            failed_number += 1
    truetrials_array.append(truetrials_number)
    stop_percentage_tot.append(stop_number/results[0]['n_trials'])
    stop_percentage_true.append(stop_number/truetrials_number)
    failed_array.append(failed_number)
    popfreqs = cbgt.collateVariable(results,'popfreqs')
    onset_values.append(onset)
    cbgt.saveResults(results, data_dir+'/onset_loop/stop_data_onsetloop_'+ '_onset_'+str(onset)+'_seed_'+str(seed),['popfreqs','popdata', 'datatables'])
    #firing_rates, q_df, performance, rt_distribution, total_performance = plt_help.extract_relevant_frames(results,seed)
    #plt_func.plot_fr_marked(firing_rates, amplitude, results[0]['stop_signal_onset'])

(pid=17479) <class 'float'>
(pid=17479) [  8  47  64 254   0 188 296 222 168  46 259 213 107 117  36  26 227 268
(pid=17479)   70 103  93 252 183 171 288  39 120 280 111  24  21 128 106  62 201 134
(pid=17479)   44 290  60 272 131  68 281  34 243 284 100 299  95 204  12 250 102  11
(pid=17479)  165  25 123  74 223  76 170  86  61  16  89 211 237 235 112 152  71  75
(pid=17479)  132 239 136 173  20 218 279 146 295  56  49 129 144 221  38 130 133  35
(pid=17479)  251 142 153 174 229 172 200  14  92 189 246 244 125 151 139   5 145 193
(pid=17479)  203  45 101 212 289  17 205 236 190  50 187  58   6  29 271 164 282 210
(pid=17479)  115 158  88 231 149  23 109 127 160 141 197  94  79 150 275  42  40  33
(pid=17479)  113 285 234 215 121 264 291 166 232   1 179 195  32 124  72 262 122 214
(pid=17479)  274 249 155  99  10 263 286 209  67   4  90 108 225 255 245 182  66  55
(pid=17479)  199 104 253 194 238   3 202   2 233 298 114 248  87 140 267 270 224 147
(pid=17479)  184 247 277  63 297 228 

(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(
(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(


(pid=17491) 2 4 0.161666`
(pid=17491) 2 4 0.161666`
(pid=17491) 2 6 0.32333`
(pid=17491) 2 6 0.32333`
(pid=17491) 3 5 0.161666`
(pid=17491) 3 5 0.161666`
(pid=17491) 3 7 0.32333`
(pid=17491) 12 8 1.0`
(pid=17491) 12 10 1.0`
(pid=17491) 3 7 0.32333`
(pid=17491) 12 8 1.0`
(pid=17491) 12 10 1.0`
(pid=17491) 12 12 0.13`
(pid=17491) 12 14 1.0`
(pid=17491) 12 16 1.0`
(pid=17491) 12 12 0.13`
(pid=17491) 12 14 1.0`
(pid=17491) 12 16 1.0`
(pid=17491) 12 17 0.0725`
(pid=17491) 13 9 1.0`
(pid=17491) 13 11 1.0`
(pid=17491) 12 17 0.0725`
(pid=17491) 13 9 1.0`
(pid=17491) 13 11 1.0`
(pid=17491) 13 13 0.13`
(pid=17491) 13 15 1.0`
(pid=17491) 13 16 1.0`
(pid=17491) 13 13 0.13`
(pid=17491) 13 15 1.0`
(pid=17491) 13 16 1.0`
(pid=17491) 13 17 0.0725`
(pid=17491) 14 8 1.0`
(pid=17491) 14 10 1.0`
(pid=17491) 14 16 0.8334`
(pid=17491) 15 9 1.0`
(pid=17491) 15 11 1.0`
(pid=17491) 15 16 0.8334`
(pid=17491) 0 14 1.0`
(pid=17491) 1 15 1.0`
(pid=17491) 4 0 1.0`
(pid=17491) 13 17 0.0725`
(pid=17491) 14 8 1.0`
(pi

(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(new_data)
(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(new_data)


(pid=17491) phasetimer 278
(pid=17491) globaltimer 1481
(pid=17491) gateFRs [28.         30.88888889]
(pid=17491) thresholds_crossed [1]
(pid=17491) other_action 0
(pid=17491) phasetimer 278
(pid=17491) globaltimer 1481
(pid=17491) gateFRs [28.         30.88888889]
(pid=17491) thresholds_crossed [1]
(pid=17491) other_action 0
(pid=17491) phasetimer 301
(pid=17491) globaltimer 1782
(pid=17491) trial_num 1
(pid=17491)   action
(pid=17491) 0   left
(pid=17491) 1  right
(pid=17491) chosen_action right
(pid=17491) Q_support_params.chosen_action[0] right
(pid=17491) trial_num 1
(pid=17491) TRIAL WISE Q DF left     0.5
(pid=17491) right    0.5
(pid=17491) Name: 0, dtype: object
(pid=17491) phasetimer 301
(pid=17491) globaltimer 1782
(pid=17491) trial_num 1
(pid=17491)   action
(pid=17491) 0   left
(pid=17491) 1  right
(pid=17491) chosen_action right
(pid=17491) Q_support_params.chosen_action[0] right
(pid=17491) trial_num 1
(pid=17491) TRIAL WISE Q DF left     0.5
(pid=17491) right    0.5
(pi

...

(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:195: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(new_data)
(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:195: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(new_data)


(pid=17491) phasetimer 301
(pid=17491) globaltimer 355500
(pid=17491) gateFRs [27.11111111 29.55555556]
(pid=17491) thresholds_crossed []
(pid=17491) phasetimer 301
(pid=17491) globaltimer 355500
(pid=17491) gateFRs [27.11111111 29.55555556]
(pid=17491) thresholds_crossed []
(pid=17491) phasetimer 301
(pid=17491) globaltimer 355801
(pid=17491) trial_num 299
(pid=17491)   action
(pid=17491) 0   left
(pid=17491) 1  right
(pid=17491) chosen_action stop
(pid=17491) Q_support_params.chosen_action[0] stop
(pid=17491) trial_num 299
(pid=17491) phasetimer 301
(pid=17491) globaltimer 355801
(pid=17491) trial_num 299
(pid=17491)   action
(pid=17491) 0   left
(pid=17491) 1  right
(pid=17491) chosen_action stop
(pid=17491) Q_support_params.chosen_action[0] stop
(pid=17491) trial_num 299


(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(new_data)
(pid=17491) /Users/cristina/Desktop/Stop_STN+D2/qvalues.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(pid=17491)   Q_df = Q_df.append(new_data)


(pid=17491) <class 'float'>
(pid=17491) [205 239 156 172 232  77 100 237 286 265  89  75  65 189 132  72 181   5
(pid=17491)   29  34  36   0 117 293 134 157 192  24 212 110  42  71 202 162 211   7
(pid=17491)  233 103 105  49 151 115  85 260 273  58 230 298  88  35 289 180 129 191
(pid=17491)   69  91  13 150  32 174 215 206  60  56 176 253  54 294 257 135  84 247
(pid=17491)  120 152 121 179  20  93 167 242 275 102 208  15   3 280 200 251 149 108
(pid=17491)  187  30  19  86 141 299 166 227  53 252 159 279 143 235  37 284 201 111
(pid=17491)   33  70 158 184 101 238 164 126 258 107  79 278  11  18 119 204 261   9
(pid=17491)  125 220 193 262 231 210 290 213 139 203   6  80 223 138 124   2  43  74
(pid=17491)  259 274 282 209 268 234 169  61 140  59 123  81 295 186 256 283 245  28
(pid=17491)   45 296  27 216  66 178 263 137  67 171 255 272 207 144  40 214  12 285
(pid=17491)  243  87 136  46 198   4  50 194  99 128 133   8 161  76  82 148 199  57
(pid=17491)  113 173  26 244 177  41 

RayTaskError(AccessDenied): [36mray::worker()[39m (pid=17491, ip=192.168.1.56)
  File "/opt/anaconda3/lib/python3.8/site-packages/ray/thirdparty_files/psutil/_common.py", line 447, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

[36mray::worker()[39m (pid=17491, ip=192.168.1.56)
  File "/opt/anaconda3/lib/python3.8/site-packages/ray/thirdparty_files/psutil/_common.py", line 447, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

[36mray::worker()[39m (pid=17491, ip=192.168.1.56)
  File "/opt/anaconda3/lib/python3.8/site-packages/ray/thirdparty_files/psutil/_psosx.py", line 344, in wrapper
    return fun(self, *args, **kwargs)
  File "/opt/anaconda3/lib/python3.8/site-packages/ray/thirdparty_files/psutil/_common.py", line 450, in wrapper
    return fun(self)
  File "/opt/anaconda3/lib/

In [ ]:
saveresults_vars(stop_percentage_tot, data_dir+'/onset_loop/loop_onsettime_stop_probs_tot'+'_seed'+str(seed))
saveresults_vars(stop_percentage_true, data_dir+'/onset_loop/loop_onsettime_stop_probs_true'+'_seed'+str(seed))
saveresults_vars(onset_values, data_dir+'/onset_loop/loop_onsettime_onset_values'+'_seed'+str(seed))
saveresults_vars(truetrials_array, data_dir+'/onset_loop/loop_onsettime_truetrials_number'+'_seed'+str(seed))
saveresults_vars(failed_array, data_dir+'/onset_loop/loop_onsettime_failedtrials_number'+'_seed'+str(seed))

In [ ]:
i = np.arange(0,170,10)
plt.figure(figsize=(10,4))
plt.bar([i for i in np.arange(0,110,10)], truetrials_array, width=4, align='center', tick_label=i)
#plt.axis([0, 160, 0, 300]) 
#axis([xmin,xmax,ymin,ymax])
plt.xlabel('Simulation (SSD) number')
plt.ylabel('Number of trials where decisionduration(RT)>SSD')
plt.title('Number of trials where decisionduration>SSD for each SSD simulation _ each simulation containing 300 trials')
plt.savefig('./Figures/'+'Truetrials-onsettime_loop'+'_.png')
plt.show()

In [ ]:
plt.plot(onset_values, stop_percentage_tot)
plt.xlabel('SSD')
plt.ylabel('Probability of stopping')
plt.savefig('./Figures/Onset-%_timeloop'+'_'+str(seed)+'.png')
#plt.ylim(-1,1)
plt.show()

In [ ]:
plt.plot(onset_values, stop_percentage_tot)
plt.xlabel('SSD')
plt.ylabel('Probability of stopping')
plt.savefig('./Figures/OnsetSTN-%_timeloop_zoompsico'+'_'+str(seed)+'.png')
plt.xlim(-1,55)
plt.show()

In [ ]:
seed

In [ ]:
onset_val_1 = loadresults_vars('./Data/onset_loop/loop_onsettime_onset_values_seed7574748')
onset_val_2 = loadresults_vars('./Data/onset_loop/loop_onsettime_onset_values_2_seed90378')
onset_tot =  onset_val_1 + onset_val_2

In [ ]:
onset_tot

In [ ]:
stop_percentage_1 = loadresults_vars('./Data/onset_loop/loop_onsettime_stop_probs_seed7574748')
stop_percentage_2 = loadresults_vars('./Data/onset_loop/loop_onsettime_stop_probs_2_seed90378')
stop_pr = stop_percentage_1 + stop_percentage_2

In [ ]:
true_1 = loadresults_vars('./Data/onset_loop/loop_onsettime_truetrials_number_seed7574748')
true_2 = loadresults_vars('./Data/onset_loop/loop_onsettime_truetrials_number_2_seed90378')
true_tot = true_1 + true_2

In [ ]:
true_tot


In [ ]:
plt.figure(figsize=(10,4))
plt.bar(onset_tot, true_tot, width=4, align='center', tick_label=onset_tot)
plt.xlabel('SSD number - simulation')
plt.ylabel('Number of trials where RT>SSD')
plt.title('Number of trials where decisionduration(RT)>SSD for each SSD simulation _ for each SSD value 300 trials running')
plt.savefig('./Figures/'+'Truetrials-onsettime_loop'+'_.png')
plt.show()

In [ ]:
stop_pr

In [ ]:
plt.plot(onset_tot, stop_pr, marker='.', linestyle = 'none')
plt.xlabel('SSD')
plt.ylabel('Probability of stopping')
plt.savefig('./Figures/Onset-%_timeloop'+'_tot_dotted_'+str(seed)+'.png')
#plt.ylim(-1,1)
plt.show()

## Duration loop

In [ ]:
import pickle 

def saveresults_vars(variable, prefix):
    pickle.dump(variable, open(prefix, 'wb'))
    
def loadresults_vars(prefix):
    return pickle.load(open(prefix, "rb"))

In [ ]:
stop_percentage_tot = []
stop_percentage_true = []
failed_array = []
truetrials_array = []
duration_values = []

for duration in np.arange(80,250,20):
    
    
    environment = {
        'params': None, #neuron parameters (init_params.py)
        'pops': None,   #population parameters (init_params.py)
        'receps' : None, #receptor parameters (init_params.py)
        'base' : None, #baseline stimulation parameters (init_params.py)
        'dpmns' : None, #dopamine related parameters (init_params.py)
        'd1' : None, #D1-MSNs population related parameters (init_params.py)
        'd2' : None, #D2-MSNs population related parameters (init_params.py)
        'channels' : pd.DataFrame([["left"],["right"]], columns=['action']), #action channels related parameters (init_params.py)
        #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
        #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
        'newpathways' : None, #connectivity parameters (popconstruct.py)
        'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
        'Q_df':None, #initialized Q-values df (qvalues.py)  
        'n_trials': 300, #number of trials (generateepochs.py)
        'volatility': 1, #frequency of changepoints (generateepochs.py)
        'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
        'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
        'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
        'maxstim': 0.95, # amplitude of the cortical input over base line #0.85
        'sustainedfraction': 0.75,

        #STN
        'stop_signal_present': True,
        'stop_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
        'stop_signal_amplitude': 0.43, # amplitude of the stop signal over base line
        'stop_signal_onset': 50, #in ms #antes 200
        'stop_signal_duration' : duration, 
        'stop_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                    #/"any" (channel given the stop signal is chosen randomly)
                                    # [list of channels] == subset of channels given the stop signa

        #D2STR
        'stop_signal_present_2': True,
        'stop_signal_probability_2': 1., #probability of trials that will get the stop signal / list of trial numbers
        'stop_signal_amplitude_2': 0.9, # amplitude of the stop signal over base line
        'stop_signal_onset_2': 50, #in ms #antes 200
        'stop_signal_duration_2' : duration,
        'stop_signal_channel_2': "all", #"all" (all channels are given the stop signal) 
                                    #/"any" (channel given the stop signal is chosen randomly)
                                    # [list of channels] == subset of channels given the stop signal

        'decision_channel' : '1', #'all'(all channels)
                                  # '0' : left channel considered 
                                  # '1' : right channel considered
    }                  
    results = cbgt.ExecutionManager(cores=7).run([pl]*1,[environment]*1)

    datatables = cbgt.collateVariable(results,'datatables')
    stop_number = 0
    failed_number = 0
    truetrials_number = 0
    
    for i in datatables[0].index:
        if datatables[0].iloc[i]['decisionduration'] > results[0]['stop_signal_onset']:
            truetrials_number += 1
        if datatables[0].iloc[i]['decision'] == 'stop':
            stop_number += 1
        if datatables[0].iloc[i]['decision'] != 'stop':
            failed_number += 1
    truetrials_array.append(truetrials_number)
    stop_percentage_tot.append(stop_number/results[0]['n_trials'])
    stop_percentage_true.append(stop_number/truetrials_number)
    failed_array.append(failed_number)
    popfreqs = cbgt.collateVariable(results,'popfreqs')
    duration_values.append(duration)
    cbgt.saveResults(results, data_dir+'/duration_loop/stop_data_durationloop_'+ '_duration_'+str(duration)+'_seed_'+str(seed),['popfreqs','popdata', 'datatables'])
    #firing_rates, q_df, performance, rt_distribution, total_performance = plt_help.extract_relevant_frames(results,seed)
    #plt_func.plot_fr_marked(firing_rates, amplitude, results[0]['stop_signal_onset'])

In [ ]:
saveresults_vars(stop_percentage_tot, data_dir+'/duration_loop/loop_durationtime_stop_probs_tot'+'_seed'+str(seed))
saveresults_vars(stop_percentage_true, data_dir+'/duration_loop/loop_durationtime_stop_probs_true'+'_seed'+str(seed))
saveresults_vars(onset_values, data_dir+'/duration_loop/loop_durationtime_duration_values'+'_seed'+str(seed))
saveresults_vars(truetrials_array, data_dir+'/duration_loop/loop_durationtime_truetrials_number'+'_seed'+str(seed))

In [ ]:
plt.plot(duration_values, stop_percentage_tot)
plt.xlabel('SS duration')
plt.ylabel('Probability of stopping')
plt.savefig('./Figures/OnsetSTN-%_durationloop'+'_'+str(seed)+'.png')
plt.show()